In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import shutil

In [2]:
zerodhadirectory = r'C:\Projects\Finances\Zerodha TradeBook'

In [3]:

os.listdir(zerodhadirectory)

['dividends-XCP798-2022_2023.csv',
 'ledger-2022-23.csv',
 'tradebook-2021-22.csv',
 'tradebook-2022-23.csv',
 'tradebook-2023-24.csv',
 'zerodha_trades_2024-05-20.parquet']

In [5]:
trade_files = [
            file for file in os.listdir(zerodhadirectory)
            if file.endswith(".csv") and file.startswith("trade") 
        ]


In [6]:
trade_files

['tradebook-2021-22.csv', 'tradebook-2022-23.csv', 'tradebook-2023-24.csv']

In [16]:
trade_dfs = []
for file in trade_files:
    file_path = os.path.join(zerodhadirectory , file )
    df = pd.read_csv( file_path , header = 0)
    df['trade_value'] = df['quantity'] * df['price']
    # Select the desired columns
    desired_columns = ['symbol', 'isin', 'trade_date', 'exchange', 'segment', 'trade_type', 'quantity', 'price', 'trade_value']
    df = df[desired_columns]
    trade_dfs.append(df)

# Concatenate DataFrames and store the result back in the dictionary
try :
    combined_trade_df = pd.concat(trade_dfs, ignore_index=True)
except Exception as e:
    print(f'Failed to create to Zerodha trade df ::: \n {str(e)}')



In [24]:
# Get today's date
today_date = datetime.today()

# Convert the date to a string in the format "YYYY-MM-DD"
today_date_str = today_date.strftime("%Y-%m-%d")

zerodha_file_path = f"{zerodhadirectory}/zerodha_trades_{today_date_str}.parquet"

# Write the DataFrame to a Parquet file with the specified file name
try: 
    combined_trade_df.to_parquet(zerodha_file_path)
    print(f'\n Zerodha trade Parquet written !!')
except Exception as e:
    print(f'Failed to write Zerodha parquet ::: {zerodha_file_path} \n {str(e)}')


 Zerodha trade Parquet written !!


In [25]:
def file_backup(banksdirectory):
    # backup dir and create directory if not exists
    backup_dir = os.path.join (banksdirectory,'backup_parquets')
    os.makedirs(backup_dir , exist_ok= True)

    # Find parquet files
    files = [file for file in os.listdir(banksdirectory) if file.endswith(".parquet")]

    # Move files
    for file in files:
        source = os.path.join(banksdirectory, file)
        destination = os.path.join(backup_dir, file)
        shutil.move(source, destination)
        print(f"Moved {file} to backup directory")

'C:\\Projects\\Finances\\Zerodha TradeBook/zerodha_trades_2024-05-20.parquet'

### Parquet

In [73]:
import os
import pandas as pd
zerodhadirectory = r'C:\\Projects\\Finances\\Zerodha TradeBook'
files = [file for file in os.listdir(zerodhadirectory) if file.endswith('.parquet')]
zerodha_file_path = os.path.join(zerodhadirectory, files[0])
zerodha_df = pd.read_parquet(zerodha_file_path)

In [74]:
#columns = [ 'symbol' , 'trade_date', 'trade_type', 'trade_value']


In [75]:
af = zerodha_df.groupby(['trade_date','symbol','trade_type'])[[ 'trade_value','quantity']].agg("sum").reset_index()
zerodha_pivot = pd.pivot_table( af ,index =['symbol'],values=['trade_value','quantity'], columns=['trade_type'], aggfunc='sum', fill_value=0).reset_index()
zerodha_pivot.columns
zerodha_pivot_cols  = [f"{i}_{j}" if j else f"{i}" for i , j in zerodha_pivot.columns]
#zerodha_pivot_cols
#zerodha_pivot_cols = ['_'.join(col).strip() for col in zerodha_pivot.columns.values]
zerodha_pivot.columns = zerodha_pivot_cols

In [81]:
zerodha_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   symbol            103 non-null    object 
 1   quantity_buy      103 non-null    float64
 2   quantity_sell     103 non-null    float64
 3   trade_value_buy   103 non-null    float64
 4   trade_value_sell  103 non-null    float64
dtypes: float64(4), object(1)
memory usage: 4.2+ KB


In [78]:
zerodha_pivot[zerodha_pivot['quantity_sell'] > zerodha_pivot['quantity_buy']]

,symbol,quantity_buy,quantity_sell,trade_value_buy,trade_value_sell
3,AARTIPHARM,0.0,1.0,0.00,354.85
9,ASKAUTOLTD,0.0,53.0,0.00,15768.10
12,AZAD,0.0,28.0,0.00,18566.60
19,CELLO,36.0,59.0,29969.70,48122.30
31,GOLDBEES-E,0.0,1.0,0.00,50.46
46,INDIASHLTR,10.0,40.0,5510.00,21957.60
49,IRCTC,2.0,10.0,5513.90,8390.55
65,MON100,0.0,15.0,0.00,1476.30
67,MOTHERSON,0.0,4.0,0.00,527.40
69,MSUMI,0.0,4.0,0.00,321.40


### Database

In [10]:
import sqlite3
from datetime import datetime
db_path = r'C:\Projects\Finances\database\Transactions.db'

In [11]:
connection = sqlite3.connect(db_path)
connection.execute("DELETE FROM zerodha")
zerodha_df.to_sql("zerodha",connection,if_exists="replace",index=False)
connection.commit()
connection.close()

In [15]:
np = zerodha_df.to_numpy()

In [19]:
np.cumsum()

TypeError: can only concatenate str (not "float") to str

In [22]:
d = zerodha_df.groupby('symbol')

In [25]:
for symbol, group in d:
    print (f"{symbol} - {group}")
    break

3MINDIA -       symbol          isin  trade_date exchange segment trade_type  quantity  \
503  3MINDIA  INE470A01017  2023-12-27      NSE      EQ        buy       1.0   
509  3MINDIA  INE470A01017  2023-12-27      NSE      EQ       sell       1.0   

        price  trade_value  
503  33149.95     33149.95  
509  36631.30     36631.30  


### Tax report

In [12]:
tax_file = r"C:\Projects\Finances\Zerodha TradeBook\taxpnl-XCP798-2023_2024-Q1-Q4.xlsx"

In [25]:
equity = pd.read_excel(tax_file , sheet_name='Equity')

In [13]:
a = pd.read_excel(tax_file)


In [14]:
# Loaded variable 'a' from kernel state

# Drop column: 'Unnamed: 0'
a = a.drop(columns=['Unnamed: 0','Unnamed: 13' ])

index = a[a['Unnamed: 1'] == 'Equity - Buyback'].index[0]
a = a.iloc[:index]

a = a.dropna()

# Promote the first row as column names
a.columns = a.iloc[0]

# Drop the first row from the DataFrame
a = a[1:].reset_index(drop=True)

In [10]:
a['term'] = 'intra' if a['Turnover'] > 0 elif a['Period of Holding'] > 365 then 'long' else 'short'

In [16]:
a['term'] = np.where (

    a['Turnover'] > 0 , 'intra' ,
    np.where(a['Period of Holding'] > 365 , 'long' , 'short')
)

In [18]:
lt = a[a['term']== 'long']

In [52]:
intra = a[a['term']== 'intra']

In [55]:
lt[lt['Symbol'] == '3MINDIA']

13,Symbol,ISIN,Entry Date,Exit Date,Quantity,Buy Value,Sell Value,Profit,Period of Holding,Fair Market Value,Taxable Profit,Turnover,term


In [23]:
grouped_df = lt.groupby(['Symbol'])[['Quantity','Buy Value', 'Sell Value', 'Profit']].agg('sum')

In [24]:
grouped_df['Profit'].sum()

-7617.7857

In [27]:
# Drop column: 'Unnamed: 0'
equity = equity.drop(columns=['Unnamed: 0','Unnamed: 13' ])

In [56]:
intra_index = equity[equity['Unnamed: 1'] == "Intraday"].index[0]
st_index = equity[equity['Unnamed: 1'] == "Short Term Trades"].index[0]
lt_index = equity[equity['Unnamed: 1'] == "Long Term Trades"].index[0]

In [57]:
intra_df = equity.iloc[intra_index+1:st_index]
st_df = equity.iloc[st_index+1:lt_index]
lt_df = equity.iloc[lt_index+1:]

In [58]:
intra_df = intra_df.dropna()

intra_df.columns = intra_df.iloc[0]
# Drop the first row from the DataFrame
intra_df = intra_df[1:].reset_index(drop=True)
intra_df = intra_df[['Symbol', 'Quantity', 'Buy Value', 'Sell Value', 'Realized P&L']]

In [59]:
def slice_df( df , start_index = None , end_index = None):

    if start_index is None and end_index is not None:
        return df.iloc[:end_index+1]
    elif end_index is not None and start_index is not None:
        return df.iloc[start_index +1 : end_index]
    else:
        return pd.DataFrame()

In [60]:
def process_sliced_df(df):
    df = df.dropna()

    df.columns = df.iloc[0]
    # Drop the first row from the DataFrame
    df = df[1:].reset_index(drop=True)
    df = df[['Symbol', 'Quantity', 'Buy Value', 'Sell Value', 'Realized P&L']]
    return df

In [61]:
lt_df = process_sliced_df(lt_df)

In [62]:
lt_df.sort_values(by= 'Symbol')

119,Symbol,Quantity,Buy Value,Sell Value,Realized P&L
15,AARTIIND,7,4847.0588,3252.6,-1594.4588
5,AARTIPHARMA,1,887.667,354.85,-532.817
9,APOLLOHOSP,2,9364.9,10005.3,640.4
2,ASIANPAINT,3,9824.95,9660.6,-164.35
7,BAJFINANCE,2,12940,15399.5,2459.5
1,CAMS,4,10076,8891.6,-1184.4
12,CLEAN,3,5199.1,3906.15,-1292.95
21,GULFOILLUB,1,517.55,613,95.45
18,HDFCAMC,5,10937.3,13593.75,2656.45
20,HDFCBANK,16,24123.25,22736,-1387.25


In [65]:
grouped_df['Profit'].sum()

-7617.7857

In [64]:
pd.merge(lt_df , grouped_df, on = 'Symbol' , suffixes= ('_eq','_tr'))

,Symbol,Quantity_eq,Buy Value_eq,Sell Value_eq,Realized P&L,Quantity_tr,Buy Value_tr,Sell Value_tr,Profit
0,PERSISTENT,2,8704.4,9927,1222.6,2,8704.4,9927.0,1222.6
1,CAMS,4,10076,8891.6,-1184.4,4,10076.0,8891.6,-1184.4
2,ASIANPAINT,3,9824.95,9660.6,-164.35,3,9824.95,9660.6,-164.35
3,TARSONS,9,6120.9,4905,-1215.9,9,6120.9,4905,-1215.9
4,SETFNIF50,41,7259.2,8441.49,1182.29,41,7259.2,8441.49,1182.29
5,AARTIPHARMA,1,887.667,354.85,-532.817,1,887.667,354.85,-532.817
6,WHIRLPOOL,5,10049.1,7230.8,-2818.3,5,10049.1,7230.8,-2818.3
7,BAJFINANCE,2,12940,15399.5,2459.5,2,12940.0,15399.5,2459.5
8,HINDUNILVR,6,14301.3,15060.95,759.65,6,14301.3,15060.95,759.65
9,APOLLOHOSP,2,9364.9,10005.3,640.4,2,9364.9,10005.3,640.4
